In [29]:
import pandas as pd
import tabulate_outputs_and_folders as toaf
import argparse
from importlib import reload

In [53]:
reload(toaf)    

args = toaf.parser.parse_args(
    ['cu_nh3/standard/metadata.json', '--outtype', 'csv']
    )
#args = toaf.parser.parse_args(['cu_nh3/xg/metadata.json'])
df = toaf.main(args)

| ARGUMENTS PROVIDED
metadata_path        cu_nh3/standard/metadata.json
enertypes            None
outtype              csv
calc_type            standard
template             standard_all_gamma.tinp
file_prefix          standard_all_gamma
bases
   iterable             True
   subfolder            True
   values               ['aug-cc-pVDZ', 'aug-cc-pVTZ', 'aug-cc-pVQZ']
   prefix               {value}
distances
   iterable             True
   subfolder            False
   values               [1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.5, 2.7, 3.0, 3.5, 4.0, 5.0, 9.0, 9999.0]
   format               06.3f
   prefix               _r_{value}
reference_outputs    ['standard_all_gamma_r_09.000']


In [54]:
df

,bases,distances,GEM_BETA,TOT_ENER,REF_ENER,CORR_ENER,outfile
0,aug-cc-pVDZ,01.500,0.5,-1695.555068,-1694.876489,-0.678579,cu_nh3/standard/aug-cc-pVDZ/standard_all_gamma...
1,aug-cc-pVDZ,01.500,0.6,-1695.593714,-1694.876489,-0.717225,cu_nh3/standard/aug-cc-pVDZ/standard_all_gamma...
2,aug-cc-pVDZ,01.500,0.7,-1695.624015,-1694.876489,-0.747526,cu_nh3/standard/aug-cc-pVDZ/standard_all_gamma...
3,aug-cc-pVDZ,01.500,0.8,-1695.647860,-1694.876489,-0.771371,cu_nh3/standard/aug-cc-pVDZ/standard_all_gamma...
4,aug-cc-pVDZ,01.500,0.9,-1695.666631,-1694.876489,-0.790142,cu_nh3/standard/aug-cc-pVDZ/standard_all_gamma...
...,...,...,...,...,...,...,...
1270,aug-cc-pVQZ,9999.000,2.5,-1695.775418,-1694.952507,-0.822912,cu_nh3/standard/aug-cc-pVQZ/standard_all_gamma...
1271,aug-cc-pVQZ,9999.000,2.6,-1695.775072,-1694.952507,-0.822565,cu_nh3/standard/aug-cc-pVQZ/standard_all_gamma...
1272,aug-cc-pVQZ,9999.000,2.7,-1695.774644,-1694.952507,-0.822137,cu_nh3/standard/aug-cc-pVQZ/standard_all_gamma...
1273,aug-cc-pVQZ,9999.000,2.8,-1695.774145,-1694.952507,-0.821639,cu_nh3/standard/aug-cc-pVQZ/standard_all_gamma...


In [13]:
filtered_df = df[df['distances'] == '9999.000']
filtered_df

,bases,gamma_set,distances,total energy,correlation energy,outfile
3,aug-cc-pVDZ,2.40_1.40_2.40,9999.000,-1695.727819,-0.796418,cu_nh3/xg/aug-cc-pVDZ/gamma_set_2.40_1.40_2.40...
7,aug-cc-pVDZ,2.40_1.40_2.15,9999.000,-1695.727819,-0.796418,cu_nh3/xg/aug-cc-pVDZ/gamma_set_2.40_1.40_2.15...
11,aug-cc-pVDZ,2.40_1.40_1.90,9999.000,-1695.727819,-0.796418,cu_nh3/xg/aug-cc-pVDZ/gamma_set_2.40_1.40_1.90...
15,aug-cc-pVDZ,2.40_1.40_1.65,9999.000,-1695.727819,-0.796418,cu_nh3/xg/aug-cc-pVDZ/gamma_set_2.40_1.40_1.65...
19,aug-cc-pVDZ,2.40_1.40_1.40,9999.000,-1695.727819,-0.796418,cu_nh3/xg/aug-cc-pVDZ/gamma_set_2.40_1.40_1.40...
23,aug-cc-pVTZ,2.40_1.40_2.40,9999.000,-1695.771845,-0.824562,cu_nh3/xg/aug-cc-pVTZ/gamma_set_2.40_1.40_2.40...
27,aug-cc-pVTZ,2.40_1.40_2.15,9999.000,-1695.771845,-0.824562,cu_nh3/xg/aug-cc-pVTZ/gamma_set_2.40_1.40_2.15...
31,aug-cc-pVTZ,2.40_1.40_1.90,9999.000,-1695.771845,-0.824562,cu_nh3/xg/aug-cc-pVTZ/gamma_set_2.40_1.40_1.90...
35,aug-cc-pVTZ,2.40_1.40_1.65,9999.000,-1695.771845,-0.824562,cu_nh3/xg/aug-cc-pVTZ/gamma_set_2.40_1.40_1.65...
39,aug-cc-pVTZ,2.40_1.40_1.40,9999.000,-1695.771845,-0.824562,cu_nh3/xg/aug-cc-pVTZ/gamma_set_2.40_1.40_1.40...


In [11]:
df['distances'].values

array(['01.900', '02.000', '02.100', '9999.000', '01.900', '02.000',
       '02.100', '9999.000', '01.900', '02.000', '02.100', '9999.000',
       '01.900', '02.000', '02.100', '9999.000', '01.900', '02.000',
       '02.100', '9999.000', '01.900', '02.000', '02.100', '9999.000',
       '01.900', '02.000', '02.100', '9999.000', '01.900', '02.000',
       '02.100', '9999.000', '01.900', '02.000', '02.100', '9999.000',
       '01.900', '02.000', '02.100', '9999.000'], dtype=object)

In [18]:
std_nh3 = (1.4,   -0.26201805)
std_cu = (2.4,   -0.56254499)
std_nh3[1] + std_cu[1]

-0.82456304